<a href="https://colab.research.google.com/github/josepmatias/ONE/blob/main/auto_campanhas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
pip install pyxlsb

In [3]:
#nomear os ficheiros

#activacoes="auto_listagemactivacoes.xlsb"
#auto="03.2024 - Auto Upsell_Prospect_Rec_Cancelados_formulas.xlsb"
modelo_upfront="modelo upfront.xlsb"
activacoes="ONE_Ativações_Serviços_Marcações_Migrações_mar24.xlsb"

In [11]:
#carregar ficheiro de activações e auto

from google.colab import files
uploaded = files.upload()
uploaded2=files.upload()

In [ ]:
#from google.colab import files
#uploaded3 = files.upload()

Saving modelo upfront.xlsb to modelo upfront.xlsb


In [5]:
#ler ficheiro db_upfront
#db_upfront usada para o valor upfront €, excepto quando análise daf é "DTH para Cabo" e "Pré-Pago"
db_upfront=pd.read_excel(modelo_upfront, sheet_name="modelo upfront", header=1).iloc[:, [2, 3, 4]]

db_upfront


,Sub-Área de Negócio,Upfront,Pts GCO
0,ALARME,25.0,depende da campanha
1,CABO,10.0,depende da campanha
2,Desporto,2.0,0
3,DTH,10.0,depende da campanha
4,DTH para Cabo,5.0,1.25
5,IOT,3.0,depende da campanha
6,Marcações,1.5,depende da campanha
7,Migrações Stand Alone,2.5,depende da campanha
8,NAVEGAÇÃO SEGURA,0.5,0
9,NET,10.0,depende da campanha


In [6]:
#ler ficheiro db_pontos_gco
#db_pontos_gco é usada para fixar pontos (coluna pts GCO) excepto quando a análise daf é "Comissionada Navegação Segura", "Comissionado Seguro" \
#"Comissionado Router 6.0", ""Comissionado Power wi-fi""
db_pontos_gco=pd.read_excel(modelo_upfront, sheet_name="modelo upfront", header=1).iloc[:, 8:10]
db_pontos_gco.head()

,Campanha,pts GCO
0,lm_churn,0.75
1,lm_churn_ac,0.75
2,prospect_alarmes,3.00
3,prospect_blitz_blm,2.00
4,prospect_blitz_wow,2.00


In [9]:
#ler ficheiro da listagem de activações
auto_db=pd.read_excel(activacoes, sheet_name="Detalhe")
print(auto_db.shape)
auto_db.head()

(8779, 47)


,Mês de Fecho,Order Item,Chave,SFID,Comissionamento,Sub-Área de Negócio,Service Account,Customer Account (SA),CA&UA,Alojamento Id,...,Notas DS,Upfront €,pts GCO,GCO €,Total €,Final,Analise DAF,Pagar,Conclusão SDD,Análise SDD
0,45352,1-KLM2AVR,40168940583780086295,L.G01.UP1.043,ONE,CABO,S965395918,C145652802,C1456528027901704365,8996880.0,...,Uso da campanha final ou wise dará pts de GCO ...,10.0,3.00,2.0,12.0,ok,Comissionado,Sim,NaN,NaN
1,45352,1-KNXHCJ1,40168940583780047659,L.G01.UP1.011,ONE,CABO,S825253392,C172136602,C1721366021255016819,2122847.0,...,NaN,10.0,3.00,2.0,12.0,ok,Comissionado,Sim,NaN,NaN
2,45352,1-KMQRCQB,20544240583580122712,B.C01.UP1.287,ONE,CABO,S846897051,C173519001,C1735190017914773340,2244259.0,...,NaN,10.0,3.00,2.0,12.0,ok,Comissionado,Sim,NaN,NaN
3,45352,1-KLG8LET,40168940583770080648,L.G01.UP1.627,ONE,CABO,S965388611,C174106801,C17410680110012563678,4146708.0,...,NaN,10.0,2.00,-7.5,2.5,ok,Comissionado Blitz BLM,Sim,NaN,NaN
4,45352,1-KOOQNHS,40168941003680079762,L.G01.RMC.006,ONE,CABO,S848815192,C225385001,C22538500110016569851,1565886.0,...,NaN,5.0,1.25,1.0,6.0,ok,Comissionado DTH para Cabo,Sim,NaN,NaN


In [75]:
#processo de cálculo de upfront para operador
def upfront (db, db2, str1="DTH para Cabo", str2="Pré-Pago"):

  db_trad=db.copy()
  for cont in range(3):
    if cont==0:
      indice=db.loc[(db["Pagar"]=="Sim") & (db["Analise DAF"].str.find(str1)==-1) & (db["Analise DAF"].str.find(str2)==-1)].index
      area_negocio=db.loc[indice, "Sub-Área de Negócio"].unique()
      dici={}
      for i in area_negocio:
        dici[i]=db2.loc[db2.iloc[:, 0]==i, "Upfront"].tolist()[0]
      db_trad.loc[indice, "upfront"]=db_trad.loc[indice, "Sub-Área de Negócio"].map(dici)
    elif cont==1:
      indice=db.loc[(db["Pagar"]=="Sim") & (db["Analise DAF"].str.find(str1)!=-1)].index
      area_negocio2=db.loc[indice, "Sub-Área de Negócio"].apply(lambda i: i+str1)
      for i in area_negocio2.unique():
        dici[i]=db2.loc[db2.iloc[:, 0]==i, "Upfront"].tolist()[0]
      db_trad.loc[indice, "upfront"]=db_trad.loc[indice, "Sub-Área de Negócio"].apply(lambda i: i+str1).map(dici)
    elif cont==2:
      indice=db.loc[(db["Pagar"]=="Sim") & (db["Analise DAF"].str.find(str2)!=-1)].index
      area_negocio3=db.loc[indice, "Sub-Área de Negócio"].apply(lambda i: i+str2)
      for i in area_negocio3.unique():
        dici[i]=db2.loc[db2.iloc[:, 0]==i, "Upfront"].tolist()[0]
      db_trad.loc[indice, "upfront"]=db_trad.loc[indice, "Sub-Área de Negócio"].apply(lambda i: i+str2).map(dici)
  return db_trad[["Sub-Área de Negócio", "Analise DAF", "upfront"]]

db_up=upfront(auto_db, db_upfront)

In [116]:
#"Comissionada Navegação Segura", "Comissionado Seguro", "Comissionado Router 6.0", "Comissionado Power wi-fi" -> análise daf
no_points=["Comissionada Navegação Segura", "Comissionado Seguro", "Comissionado Router 6.0", "Comissionado Power wi-fi"]
def pontos_op(db, db2, lista=no_points, col="Campanha wise"):
  db_pontos=db.copy()

  for j in range(2):
    if j==0:
      indice=db.loc[(db["Pagar"]=="Sim") & ~(db["Analise DAF"].isin(lista)), col].index
      #retirar string "Info comercial: " e lidar com caracter estranho
      campanhas=db.loc[(db["Pagar"]=="Sim") & ~(db["Analise DAF"].isin(lista)), col].str.replace("Info comercial: ", "").unique().tolist()
      campanhas.remove("0x2a")

      #encontrar os pts para a campanha
      dici={}
      for i in campanhas:
        dici[i]=db2.loc[db2.iloc[:, 0]==i, "pts GCO"].tolist()[0]

      #criar coluna pontos a partir da campanha wise
      db_pontos.loc[indice, "pontos"]=db.loc[indice, col].map(dici)

  #colocar pontos da lista a zero
    else:
      indice=db.loc[(db["Pagar"]=="Sim") & (db["Analise DAF"].isin(lista)), col].index

      db_pontos.loc[indice, "pontos"]=0

  return db_pontos

db_pontos=pontos_op(auto_db, db_pontos_gco)

In [121]:
#[i for i in campanhas]
#no_points=["Comissionada Navegação Segura", "Comissionado Seguro", "Comissionado Router 6.0", "Comissionado Power wi-fi"]
#lista=auto_db.loc[(auto_db["Pagar"]=="Sim") & ~(auto_db["Analise DAF"].isin(no_points)), \
#"Campanha wise"].str.replace("Info comercial: ", "").unique().tolist()



print(db_pontos["pontos"].value_counts())


print(db_pontos.loc[db_pontos["pontos"]==3.00, "Campanha wise"].unique())
db_pontos.loc[db_pontos["pontos"]==2.00, "Campanha wise"].unique()

db_pontos.loc[db_pontos["pontos"]==1.25, "Campanha wise"].unique()

pontos
1.25    5216
2.00    1870
0.75     920
3.00     624
0.00      78
Name: count, dtype: int64
['prospect_pos' 'prospect_m_c' 'prospect_pre' 'prospect_pos_wtf'
 'prospect_diversos']


array(['up_migracao_dth', 'up_1p_pf', 'up_1p', 'up_pos_pago', 'up_wtftop'],
      dtype=object)

In [10]:
#db_pontos.loc[db_pontos["pontos"].isna()==True, ["pontos", "Analise DAF", "Campanha wise"]]

#auto_db["Campanha wise"].unique()

auto_db.loc[auto_db["Campanha wise"]=="0x2a", "Order Item"]

100     1-KN32U22
168     1-KJZOQEW
511     1-KJZR4VT
3307    1-KN32U6X
3376    1-KJZR4X9
Name: Order Item, dtype: object